In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [43]:
data = pd.read_csv('dataset_demanda_df_completo.csv')

In [44]:
data.head()

,zona,data,faixa_horaria,tipo_ocorrencia,clima,evento_especial,denuncias_app_ultimas_24h,anomalias_cameras_ultimas_24h,risco_ocorrencia,volume_estimado
0,Arniqueira,2025-06-24,00-06h,Roubo,Chuva,Não,3,15,69,6
1,Sol Nascente/Pôr do Sol,2025-07-07,18-23h,Todos,Nublado,Não,25,14,55,3
2,Itapoã,2025-07-18,12-18h,Furto,Chuva,Sim,19,9,77,0
3,Riacho Fundo,2025-07-20,12-18h,Todos,Céu limpo,Sim,5,5,65,7
4,Candangolândia,2025-06-23,18-23h,Roubo,Nublado,Sim,9,5,83,9


In [45]:
data = data.drop('volume_estimado', axis=1)

## Engenharia de variaveis para modelo de previsão de demanda

In [48]:
def recomendar_viaturas_avancado(risco, denuncias, anomalias):
    score = (risco * 0.6) + (denuncias * 0.2) + (anomalias * 0.2)

    if score <= 20:
        return 1
    elif score <= 35:
        return 2
    elif score <= 40:
        return 3
    elif score <= 60:
        return 4
    elif score <= 80:
        return 4
    elif score <= 100:
        return 5
    else:
        return 6


data['viaturas_recomendadas'] = data.apply(
    lambda row: recomendar_viaturas_avancado(
        row['risco_ocorrencia'],
        row['denuncias_app_ultimas_24h'],
        row['anomalias_cameras_ultimas_24h']
    ),
    axis=1
)


In [49]:
data['data'] = pd.to_datetime(data['data'])

# Dia da semana (0 = segunda, 6 = domingo)
data['dia_da_semana'] = data['data'].dt.dayofweek

# Final de semana (sábado ou domingo)
data['final_de_semana'] = data['dia_da_semana'].apply(lambda x: 1 if x >= 5 else 0)

# Mês
data['mes'] = data['data'].dt.month

# Semana do mês
data['semana_do_mes'] = data['data'].apply(lambda x: (x.day - 1) // 7 + 1)

# Feriado (exemplo simples, você precisa criar sua lista de feriados reais)
lista_feriados = ['2025-01-01', '2025-04-21', '2025-09-07', '2025-10-12', '2025-11-15']  # Exemplo Brasil
lista_feriados = pd.to_datetime(lista_feriados)
data['feriado'] = data['data'].isin(lista_feriados).astype(int)

# Tendência de risco nos últimos 3 dias (média móvel de 3 dias por zona + tipo de ocorrência)
data = data.sort_values(['zona', 'tipo_ocorrencia', 'data'])
data['media_3dias'] = data.groupby(['zona', 'tipo_ocorrencia'])['risco_ocorrencia'] \
                          .transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# Tendência de risco nos últimos 7 dias (média móvel de 7 dias por zona + tipo de ocorrência)
data['media_7dias'] = data.groupby(['zona', 'tipo_ocorrencia'])['risco_ocorrencia'] \
                          .transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# Crescimento percentual de risco em relação aos 7 dias anteriores
data['media_7dias_anterior'] = data.groupby(['zona', 'tipo_ocorrencia'])['risco_ocorrencia'] \
                                   .transform(lambda x: x.shift(1).rolling(window=7, min_periods=1).mean())

data['crescimento_percentual'] = ((data['media_7dias'] - data['media_7dias_anterior']) /
                                  (data['media_7dias_anterior'] + 1e-6)) * 100

### Média Diária, Mensal

In [50]:
import pandas as pd

# Calcular média diária e mensal
data['diaria_avg'] = data.groupby(['tipo_ocorrencia', 'zona', 'data'])['risco_ocorrencia'].transform('mean')
data['mensal_avg'] = data.groupby(['tipo_ocorrencia', 'zona', 'data'])['risco_ocorrencia'].transform('mean')

# Calcular média móvel de 7 dias
data['media_movel_7dias'] = data.groupby(['tipo_ocorrencia', 'zona'])['risco_ocorrencia'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# Média por local e tipo de demanda
data['media_risco_por_zona_ocorrencia'] = data.groupby(['zona', 'tipo_ocorrencia'])['risco_ocorrencia'].transform('mean')
data['media_risco_por_zona'] = data.groupby(['zona'])['risco_ocorrencia'].transform('mean')
data['media_risco_por_ocorrencia'] = data.groupby(['tipo_ocorrencia'])['risco_ocorrencia'].transform('mean')

In [51]:
data = data.dropna()

In [52]:
data.head()

,zona,data,faixa_horaria,tipo_ocorrencia,clima,evento_especial,denuncias_app_ultimas_24h,anomalias_cameras_ultimas_24h,risco_ocorrencia,viaturas_recomendadas,...,media_3dias,media_7dias,media_7dias_anterior,crescimento_percentual,diaria_avg,mensal_avg,media_movel_7dias,media_risco_por_zona_ocorrencia,media_risco_por_zona,media_risco_por_ocorrencia
6391,Arniqueira,2025-06-21,12-18h,Furto,Nublado,Não,24,15,0,1,...,45.000000,45.000000,90.000000,-49.999999,45.0,45.0,45.000000,48.03125,49.512915,49.102742
435,Arniqueira,2025-06-22,00-06h,Furto,Chuva,Sim,14,7,16,1,...,35.333333,35.333333,45.000000,-21.481481,16.0,16.0,35.333333,48.03125,49.512915,49.102742
3658,Arniqueira,2025-06-23,18-23h,Furto,Chuva,Não,13,12,98,4,...,38.000000,51.000000,35.333333,44.339621,59.0,59.0,51.000000,48.03125,49.512915,49.102742
3995,Arniqueira,2025-06-23,12-18h,Furto,Céu limpo,Sim,23,3,26,2,...,46.666667,46.000000,51.000000,-9.803921,59.0,59.0,46.000000,48.03125,49.512915,49.102742
4834,Arniqueira,2025-06-23,00-06h,Furto,Nublado,Não,1,12,51,2,...,58.333333,46.833333,46.000000,1.811594,59.0,59.0,46.833333,48.03125,49.512915,49.102742


## Modelo

In [53]:
X = data.drop(['risco_ocorrencia','viaturas_recomendadas','data'], axis=1)
y = data[['risco_ocorrencia','viaturas_recomendadas']]

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = XGBRegressor(n_estimators=500, max_depth=10, learning_rate=0.05, random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [54]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)
y_test = np.array(y_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Mean Absolute Error (MAE): 6.920919895172119
Mean Squared Error (MSE): 145.74305725097656
Root Mean Squared Error (RMSE): 12.072408924940232
R-squared (R²): 0.6338194608688354


## Salvar o modelo

In [55]:
import pickle
import gzip

with gzip.open('model_cidade_segura.pkl.gz', 'wb') as f:
    pickle.dump(model, f)

In [56]:
data.to_csv('Dados_tratados.csv', index = False)